# Remaining issues

## Only a subset of mathematic functions

Let's take this example : we want to compute the total energy `e` of an electron whose linear momentum `p` is known. We can deduce it from the equality `e^2 = m^2*c^4 + p^2*c^2`, with `c` the speed of light and `m` the mass of the electron.

In [1]:
%%file tmp.remaining-issues.cpp

#include <iostream>
#include "phys/units/io.hpp"
#include "phys/units/quantity.hpp"

using namespace phys::units ;
using namespace phys::units::io ;
using namespace phys::units::literals ;

int main()
 {
  constexpr auto m = 9.109e-31_kg ;
  constexpr auto c = 299792458_m/second ; 
  constexpr auto p = 2e-22_kg*meter/second ;

  auto e = sqrt(square(m)*nth_power<4>(c)+square(p)*square(c)) ;  
       
  std::cout << e << std::endl ;
 }

Overwriting tmp.remaining-issues.cpp


In [2]:
!rm -f tmp.remaining-issues.exe && g++ -I. -std=c++17 tmp.remaining-issues.cpp -o tmp.remaining-issues.exe

In [3]:
!./tmp.remaining-issues.exe

1.01476e-13 J


We were not able to use `std::pow`, which would have not correctly handle the unit of `c`.

We were not able to benefit for the `constexpr` of `e`,  because the `sqrt` function used is not provided by the standard library, but is an alternative implementation from PhysUnits, which can handle units, but is not a `constexpr`.

The use of a library of units tends to **limit us to the numerical functions provided by this library**, with their limitations.

## Irrelevant internal reference unit 

Let's try our example with `float`.

In [4]:
%%file tmp.remaining-issues.cpp

#include <iostream>

#include "phys/units/io.hpp"
#include "phys/units/quantity.hpp"

using namespace phys::units ;
using namespace phys::units::io;
using namespace phys::units::literals ;
using momentum_d = dimensions< 1, 1, -1 > ;

int main()
 {
  constexpr quantity<mass_d,float> m = 9.109e-31_kg ;
  constexpr quantity<speed_d,float> c = 299792458_m/second ; 
  constexpr quantity<momentum_d,float> p = m*0.75*c ;

  auto e = sqrt(square(m)*nth_power<4>(c)+square(p)*square(c)) ;  
       
  std::cout << e << std::endl ;
 }

Overwriting tmp.remaining-issues.cpp


In [5]:
!rm -f tmp.remaining-issues.exe && g++ -I. -std=c++17 tmp.remaining-issues.cpp -o tmp.remaining-issues.exe

In [6]:
!./tmp.remaining-issues.exe

6.14677e-14 J


**The result if very wrong**. However, we can prove that this computation can be done with simple precision, provided we express our variables in a unit which is close to the scale of the problem, which is not the case of `kg` above, used internally by PhysUnits.

Below, the mass and linear momentum have been shift by `1e27`, any computation is enforced in `float`, and the result is a lot more acceptable.

In [1]:
%%file tmp.remaining-issues.cpp

#include <iostream>
#include <cmath>

int main()
 {
  constexpr float m = 0.0009109 ;
  constexpr float c = 299792458 ;
  constexpr float p = m*0.75f*c ;

  constexpr float m2 = m*m ; 
  constexpr float c2 = c*c ; 
  constexpr float c4 = c2*c2 ; 
  constexpr float p2 = p*p ; 
  constexpr float e = sqrt(m2*c4+p2*c2) ;
    
  std::cout << e*1e-27 << std::endl ;
 }

Writing tmp.remaining-issues.cpp


In [2]:
!rm -f tmp.remaining-issues.exe && g++ -I. -std=c++17 tmp.remaining-issues.cpp -o tmp.remaining-issues.exe

In [3]:
!./tmp.remaining-issues.exe

1.02335e-13


In the domain of infinitely small and infinitely large, it is aberrant to store values in seconds, meters, kilograms, etc. **The units library must let you control the unit used for internal storage**, or you may suffer a huge loss of precision. About this topic, Martin Moene (the author of PhysUnits) indicates the work of Tony Pilz (which I did not explore yet) :
* ScaledValue : https://github.com/tonypilz/ScaledValue
* Units : https://github.com/tonypilz/units

## Libraries of linear algebra

In scientific computing, we often use a lot of matrice computation, and deeply rely on libraries like Eigen... Can we make Eigen uses units provided by PhysUnits ? Let's try to compute the linear momentum of our electron, but in 3D space, with x/y/z coordinates.

In [4]:
%%file tmp.remaining-issues.h

#include <iostream>

#include "phys/units/io.hpp"
#include "phys/units/quantity.hpp"
#include "Eigen/Dense"

using namespace phys::units ;
using namespace phys::units::io;
using namespace phys::units::literals ;

using momentum_d = dimensions< 1, 1, -1 > ;
 
using Speed = quantity<speed_d> ;
using Momentum = quantity<momentum_d> ;

using Speed3d = Eigen::Matrix<Speed,3,1> ;
using Momentum3d = Eigen::Matrix<Momentum,3,1> ;

Writing tmp.remaining-issues.h


In [5]:
%%file tmp.remaining-issues.cpp

#include "tmp.remaining-issues.h"

int main()
 {
  constexpr quantity<mass_d> m = 0.0009109_yg ;
  constexpr quantity<speed_d> c = 299792458_m/second ;
    
  Speed speed_x = 0.75/sqrt(3.)*c ;
  Speed speed_y = speed_x, speed_z = speed_x ;
  Speed3d speed(speed_x,speed_y,speed_z) ;

  Momentum3d momentum = m*speed ;
  Momentum p = momentum.norm()*kilogram*meter/second ;
  auto e = sqrt(square(m)*nth_power<4>(c)+square(p)*square(c)) ;  
  std::cout << speed << std::endl ;
 }

Overwriting tmp.remaining-issues.cpp


In [6]:
!rm -f tmp.remaining-issues.exe && g++ -I/opt/miniconda3/include/eigen3/ -I. -std=c++17 tmp.remaining-issues.cpp -o tmp.remaining-issues.exe

In file included from /opt/miniconda3/include/eigen3/Eigen/Core:263,
                 from /opt/miniconda3/include/eigen3/Eigen/Dense:1,
                 from tmp.remaining-issues.h:6,
                 from tmp.remaining-issues.cpp:2:
/opt/miniconda3/include/eigen3/Eigen/src/Core/arch/SSE/PacketMath.h:29:39: warning: ignoring attributes on template argument ‘__m128’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wignored-attributes-Wignored-attributes]8;;]
   29 | template<> struct is_arithmetic<__m128>  { enum { value = true }; };
      |                                       ^
/opt/miniconda3/include/eigen3/Eigen/src/Core/arch/SSE/PacketMath.h:30:40: warning: ignoring attributes on template argument ‘__m128i’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wignored-attributes-Wignored-attributes]8;;]
   30 | template<> struct is_arithmetic<__m128i> { enum { value = true }; };
      |                                        ^
/opt/minicond

In file included from /opt/miniconda3/include/eigen3/Eigen/Core:277,
                 from /opt/miniconda3/include/eigen3/Eigen/Dense:1,
                 from tmp.remaining-issues.h:6,
                 from tmp.remaining-issues.cpp:2:
/opt/miniconda3/include/eigen3/Eigen/src/Core/DenseCoeffsBase.h: In instantiation of ‘class Eigen::DenseCoeffsBase<Eigen::Matrix<float, 4, 1>, 0>’:
/opt/miniconda3/include/eigen3/Eigen/src/Core/DenseCoeffsBase.h:274:7:   required from ‘class Eigen::DenseCoeffsBase<Eigen::Matrix<float, 4, 1>, 1>’
/opt/miniconda3/include/eigen3/Eigen/src/Core/DenseCoeffsBase.h:636:7:   required from ‘class Eigen::DenseCoeffsBase<Eigen::Matrix<float, 4, 1>, 3>’
/opt/miniconda3/include/eigen3/Eigen/src/Core/util/XprHelper.h:371:8:   required from ‘struct Eigen::internal::special_scalar_op_base<Eigen::Matrix<float, 4, 1>, float, float, false>’
/opt/miniconda3/include/eigen3/Eigen/src/Core/DenseBase.h:41:34:   required from ‘class Eigen::DenseBase<Eigen::Matrix<float, 4, 1> >’


In file included from /opt/miniconda3/include/eigen3/Eigen/Core:301,
                 from /opt/miniconda3/include/eigen3/Eigen/Dense:1,
                 from tmp.remaining-issues.h:6,
                 from tmp.remaining-issues.cpp:2:
/opt/miniconda3/include/eigen3/Eigen/src/Core/Dot.h: In instantiation of ‘typename Eigen::NumTraits<typename Eigen::internal::traits<T>::Scalar>::Real Eigen::MatrixBase<Derived>::norm() const [with Derived = Eigen::Matrix<phys::units::quantity<phys::units::dimensions<1, 1, -1> >, 3, 1>; typename Eigen::NumTraits<typename Eigen::internal::traits<T>::Scalar>::Real = phys::units::quantity<phys::units::dimensions<1, 1, -1> >; typename Eigen::internal::traits<T>::Scalar = phys::units::quantity<phys::units::dimensions<1, 1, -1> >]’:
tmp.remaining-issues.cpp:14:29:   required from here
/opt/miniconda3/include/eigen3/Eigen/src/Core/Dot.h:128:14: error: could not convert ‘phys::units::sqrt<phys::units::dimensions<1, 1, -1>, double>(((const Eigen::MatrixBase<Eigen:

In file included from /opt/miniconda3/include/eigen3/Eigen/Core:315,
                 from /opt/miniconda3/include/eigen3/Eigen/Dense:1,
                 from tmp.remaining-issues.h:6,
                 from tmp.remaining-issues.cpp:2:
/opt/miniconda3/include/eigen3/Eigen/src/Core/Redux.h: In instantiation of ‘typename Eigen::internal::traits<T>::Scalar Eigen::DenseBase<Derived>::sum() const [with Derived = Eigen::CwiseUnaryOp<Eigen::internal::scalar_abs2_op<phys::units::quantity<phys::units::dimensions<1, 1, -1> > >, const Eigen::Matrix<phys::units::quantity<phys::units::dimensions<1, 1, -1> >, 3, 1> >; typename Eigen::internal::traits<T>::Scalar = phys::units::quantity<phys::units::dimensions<1, 1, -1> >]’:
/opt/miniconda3/include/eigen3/Eigen/src/Core/Dot.h:115:46:   required from ‘typename Eigen::NumTraits<typename Eigen::internal::traits<T>::Scalar>::Real Eigen::MatrixBase<Derived>::squaredNorm() const [with Derived = Eigen::Matrix<phys::units::quantity<phys::units::dimensions<1, 1

Actually, Eigen does not accept to multiply a scalar with a vector if all the numbers are not expressed in the same types : **strong typing is not compatible with popular linear algebra libraries**.

Is there an easy way to convert the variables back into builtin numerical types before using such library ? And put them back into physical units afterward ?

Can we setup some sort of code generation, and use physical units during development, together with a strongly typed avatar of eigen, then move back to builtin types and Eigen for production ? **Not an easy way !**

## Also...

**I/O** : When handling very large data sets, how to add the lacking units afterwards ? How to store the data with their units efficiently ?

**Compilation time**: provided there exists a units-friendly linear algebra library, what will be the compilation time, if  every `Vector/Matrix` must be instanciated with all the possible physical units used in the program... And let's speak about the compilation error messages...

## In the end

Until there exists some linear algebra library which is compatible with strong types, do not throw the baby out with the bathwater:
* use strong types for integers: index, sizes, identifiers, ... ;
* rely on user-defined literals for the handling of physical units multipliers ;
* use physical units in code areas which do not require some incompatible external library ;
* select reference storage units at the right scale for your problem.

# Questions ?

© *CNRS 2020*  
*Assemblée et rédigée par David Chamont, cette œuvre est mise à disposition selon les termes de la [Licence Creative Commons - Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*